In [1]:
%load_ext tensorboard

import shutil


import torch
import torchvision
import torch.nn.functional as F
import torchvision.transforms as transforms

import pytorch_lightning as pl
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.metrics import functional as FM

torch.cuda.is_available()

Microsoft Visual C++ Redistributable is not installed, this may lead to the DLL load failure.
                 It can be downloaded at https://aka.ms/vs/16/release/vc_redist.x64.exe


D:\conda\envs\lightning\lib\site-packages\pytorch_lightning\utilities\distributed.py:37: UserWarning: Unsupported `ReduceOp` for distributed computing.
  warnings.warn(*args, **kwargs)


True

# LightningDataModule

In [2]:
class CIFAR10Dataset(pl.LightningDataModule):
    def __init__(self):
        super().__init__()
        self.batch_size_ = 16
        
    def prepare_data(self):
        transform = transforms.Compose([transforms.ToTensor()])
        self.train_ds_ = torchvision.datasets.CIFAR10('data/', train=True, transform=transform, download=True)
        
    def setup(self, stage=None):
        val_length = int(0.1 * len(self.train_ds_))
        train_length = len(self.train_ds_) - val_length
        self.train_ds_, self.val_ds_ = torch.utils.data.random_split(self.train_ds_, [train_length, val_length])
    
    def train_dataloader(self):
        return torch.utils.data.DataLoader(self.train_ds_, batch_size=self.batch_size_, shuffle=True)
    
    def val_dataloader(self):
        return torch.utils.data.DataLoader(self.val_ds_, batch_size=self.batch_size_, shuffle=False)
    
    def test_dataloader(self):
        transform = transforms.Compose([transforms.ToTensor()])
        dataset = torchvision.datasets.CIFAR10('data/', train=False, transform=transform, download=True)
        loader = torch.utils.data.DataLoader(dataset=dataset, batch_size=self.batch_size_, shuffle=False)
        return loader

# LightningModule

In [3]:
class ClassificationTask(pl.LightningModule):
    def __init__(self, model):
        super().__init__()
        self.lr = 0.01
        self.model = model
        
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        result = pl.TrainResult(loss)
        result.log('train_loss', loss)
        return result

    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self.model(x)
        loss = F.cross_entropy(y_hat, y)
        acc = FM.accuracy(y_hat, y)

        # loss is tensor. The Checkpoint Callback is monitoring 'checkpoint_on'
        result = pl.EvalResult(checkpoint_on=loss)
        result.log_dict({'val_acc': acc, 'val_loss': loss})
        return result

    def test_step(self, batch, batch_idx):
        result = self.validation_step(batch, batch_idx)
        result.rename_keys({'val_acc': 'test_acc', 'val_loss': 'test_loss'})
        return result

    def configure_optimizers(self):
        return torch.optim.Adam(self.model.parameters(), lr=(self.lr or self.learning_rate))

In [4]:
model   = ClassificationTask(model=torchvision.models.mobilenet_v2(pretrained=False, progress=False))
dataset = CIFAR10Dataset()

# Tensorboard

In [5]:
try:
    shutil.rmtree('lightning_logs')
except FileNotFoundError:
    pass

%tensorboard --logdir lightning_logs

Reusing TensorBoard on port 6006 (pid 17340), started 1:47:26 ago. (Use '!kill 17340' to kill it.)

# Find LR

In [11]:
logger = TensorBoardLogger(save_dir='lightning_logs', version=1, name='lightning_logs')
trainer = pl.Trainer(gpus=1, max_epochs=10, logger=logger, auto_lr_find=True)

lr_finder = trainer.lr_find(model, dataset)

fig = lr_finder.plot(suggest=True)
fig.show()

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]


Files already downloaded and verified



  | Name  | Type        | Params
--------------------------------------
0 | model | MobileNetV2 | 3 M   


Saving latest checkpoint..


ModuleNotFoundError: No module named 'matplotlib'

# Training

In [ ]:
model.lr = lr_finder.suggestion()
trainer.fit(model, dataset)

In [ ]:
trainer.test(model, ckpt_path='best', verbose=True, datamodule=dataset)